# Reconciliations with Fuzzy Matching

In [30]:
%matplotlib inline
import fuzzywuzzy
from fuzzywuzzy import process, fuzz
import csv
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from pprint import pprint
from dateutil import parser
import re
import itertools

## look at comprehensions dictionary set can filter lists
## look at itertools combinations() for the all on all comparisons

- Code for reconciliations of free text transcript fields.  Have defined a number of fields and a few different methods for each field type. First code for reading in the file.

In [31]:
def read_clean_data(file_name):
    data = {}
    with open(file_name, 'r') as in_file:
        reader = csv.DictReader(in_file)
        headers = reader.fieldnames
        for row in reader:
            key = row['filename']
            #print(key)
            xscripts = data.get(key, [])
            #print(xscripts)
            xscripts.append(row)
            data[key] = xscripts
            #if reader.line_num >10: 
            #    break
        print(reader.line_num)
    return data, headers

In [69]:
data,headers = read_clean_data('Brit_41transcriptsforhabitat.csv')
pprint(headers)
print(len(data))
#pprint(data['BRIT118000'])

142
['',
 '',
 'subject_id',
 'filename',
 'user_name',
 'created_at',
 'Collected by',
 'Collection date',
 'Collector Number',
 'Country',
 'County',
 'Habitat',
 'Description',
 'Species',
 'State',
 'Skip',
 '',
 '']
41


- **Scientific Name:**  
    - _Method_: Fuzzy match on tokens in order  (fuzz_set_ratio) -- Will take a subset as a match if one entirely exists in the other 
    
    
    - a. if 100 match on tokens for all 3 - full agree --> 2. 
    - b. if 100 match on 2 of 3 - maj rule --> 2.
    - c. If <100 match on tokens, report two highest scoring and flag --> Done.

- 2.To deal with varieties and hybrids (x in plant data) report the longest (string or token umber?) 

    -only issue with this is getting extraneous information (e.g. 'Hypericum stragulum A. & R.' '(Hypericum  hypericoides) Jesse M. shaver'
  
    - possibly remove parenthesies?


In [75]:
FLAGS = ['all agree', 'Majority Rules','only a single transcript','more than 1 unique answer','all are blank']

def recon_scientific_name(data,field):
    flags={}
    allagree=0
    notallagree=0
    for key, xscripts in data.items():
        ### first determine if any are blank and add the non blank to a list to compare. 
        my_len = len(xscripts)
        lst=[]
        for x in range(my_len):
            if xscripts[x][field]:
                lst.append(xscripts[x][field])
        my_newlen = len(lst)
    #    print(key,"The original length %s new length %s." %(my_len,my_newlen))
    #   find the ones that are all blank and flag them
        if not len(lst):
        #   totally empty add empty flag
            print(key,"Zero Loop.The length %s new length %s." %(my_len,my_newlen))
        elif len(lst) == 1:
        #   Only have one add one flag and take what we have but make sure not empty
            print(key,"One Loop.The length %s new length %s." %(my_len,my_newlen))
        else:
            my_scorelst=[] 
            print(key,"There are %d filled in" %(my_newlen))
            for x in range(my_newlen):
                for y in range(x+1,my_newlen):
                #    print(lst[x],lst[y])
                    score = fuzz.token_set_ratio(lst[x],lst[y])
                #    print(score)
                    my_scorelst.append(score)
        hundreds=0
        for i in range(len(my_scorelst)):
            if my_scorelst[i] == 100:
                hundreds+=1
            #    print(hundreds)
        if len(my_scorelst) == hundreds:  #all scores eq 100
        #    flags[key] = dict(flag=FLAGS[0])
            allagree+=1
        #    print(key,"All agree num hundreds %d equals length %d." %(hundreds,(len(my_scorelst))))
            ## get length of each of the strings and keep the longest
            my_biglen=0
            for x in range(my_len):
                name=xscripts[x][field]
                ln = len(name)
            #    print("the length of %s is %s." %(name,ln))
                if ln > my_biglen:
                    keep=name
                    my_biglen = ln
                #    dict(ScienName[key]:name)
                #   Need a dictionary of dictionaries? key by subject and the one to keep plus a flag?
        #    print(key,keep,my_biglen)
        else:
            print(key,hundreds)
            notallagree+=1
            if hundreds == 1:
                print(key, "There is only one hundred")
                for x in range(my_newlen):
                    for y in range(x+1,my_newlen):
                    #    print(lst[x],lst[y:q])
                        score = fuzz.token_set_ratio(lst[x],lst[y])
                        if score == 100:
                            
#    print(score)
                #    my_scorelst.append(score)

    print(allagree,notallagree)


In [76]:
test = recon_scientific_name(data,'Species')
pprint(data['BRIT118130'])
#pprint(flags['BRIT118130'])

BRIT118088 There are 4 filled in
BRIT118088 3
BRIT118130 There are 2 filled in
BRIT118136 There are 4 filled in
BRIT118181 There are 3 filled in
BRIT118158 There are 3 filled in
BRIT118159 There are 3 filled in
BRIT118095 There are 3 filled in
BRIT118115 There are 4 filled in
BRIT118160 There are 3 filled in
BRIT118141 There are 3 filled in
BRIT118152 There are 3 filled in
BRIT118009 There are 3 filled in
BRIT118009 0
BRIT118087 There are 4 filled in
BRIT118000 There are 3 filled in
BRIT118015 There are 3 filled in
BRIT118171 There are 3 filled in
BRIT118155 There are 3 filled in
BRIT118134 There are 3 filled in
BRIT118134 1
BRIT118134 There is only one hundred
BRIT118122 There are 3 filled in
BRIT118122 0
BRIT118119 There are 3 filled in
BRIT118119 1
BRIT118119 There is only one hundred
BRIT118051 There are 3 filled in
BRIT118111 There are 3 filled in
BRIT118039 There are 4 filled in
BRIT118007 There are 3 filled in
BRIT118116 There are 4 filled in
BRIT118116 3
BRIT118011 There are 3 

- **Collected By:**  
    - _Method_: fuzz.token_sort_ratio() will deal with out of order names and allows for order difference. Fuzzy wuzzy deals will flag punctuation differences (e.g.  A.I  and AI are the same token), just NEED TO CHECK that it deals with capitalization.
    
    
    - a. if 100 match on tokens for all 3 - full agree - show original form.
    - b. if 100 match on 2 of 3 - maj. rule - show original form
    - c. if less than 100 match on token, [report two highest scoring in original form] 
    

In [ ]:
#def collected_by():

- **Habitat and Description:** 

    -**General Idea**:  _Want the label exactly as written._ 
        Instructions to transcribers were to write the label exactly as is. Therefore the first step is to look for exact matches. 
        If there is not an exact match want to maximize information while reducing interpretations. Generally if people do not write what is on the label that is because they have expanded an abbreviation (e.g. hwy --> highway) therefore we want the shortest string length. However, we want the labels with the longest token length to ensure that all words are accounted for.
        Main issues with these fields is that the inforamtion often can be found in both fields. For example habitat information writted in the description column and vice versa. Therefore we will first find the best label within each category and then compare across categories.
    
    -**Within Category Method**: _fuzz.token\_set\_ratio_  THEN _fuzz.token_sort_ratio_ -- first looking to see if all the words in one are present in another - would give 100 match. Second look to see the score of the sort ratio - checks the order.
    
       - a. if all 100s on the set then check on the sort and if 100 choose that comparison. --> go to 2.
       - b. if not all 100s find  -- NEED MORE LOGIC HERE>

       2. do an exact match.  --> Use the one with the shortest string length and longest word length if there is a difference.  
       
    -**Across Category Method**:  Possibly dont do this. There are reasons why we might want information in both categories..
        - THEN just taking frequency based approach for both habitat and location and secondarily - ask if location and habitat match too much, significant overlap.


In [13]:
#pprint(data['BRIT118113'])
test = data['BRIT118160']
#pprint(test)
#lst=[]
l = len(test)
pprint(l)
n=len(test)
pprint(n)
lst=[]
for x in range(n):
    for y in range(x+1,n):
        score = fuzz.token_set_ratio(test[x]['Habitat'],test[y]['Habitat'])
        ### for all 100 take the one with the more words
        # if there are different lengths report the flag , most likely there was a shorter transcript in this set. 
        # 
        #score2 = fuzz.token_sort_ratio(test[x]['Habitat'],test[y]['Habitat'])
        #print(score,x,y)
        lst.append((score,x,y))
        
slst=sorted(lst,key=lambda x: x[0], reverse=True)

pprint(slst)

3
3
[(100, 0, 1), (100, 0, 2), (100, 1, 2)]


In [50]:
def reconcile_text_fields(file_name,field):
    for key, xscripts in data.items():
        l = len(xscripts)
        #for x in range(l-1):
            #if not xscripts[l][field]:
            #    print(xscripts[l][field])
            #    flag='all blank'
            #    print(xscripts[field])
        #counts = Counter([xscripts[field] for x in xscripts if x[field] and x[field].lower()])
        #if not len(counts):
        #    flags[key] = dict(flag=FLAGS[4], value='', top_count=0, blank_count=len(xscripts))
        #l = len(xscripts)
        for x in range(n):
            for y in range(x+1,n):
                score = fuzz.token_set_ratio(xscripts[x][field],xscripts[y][field])
                print(key,score,x,y)
        #break
    #    n = len(test)
    #    pprint(test)
    return(key)
    #return (scores,comparisons)

In [51]:
test = reconcile_text_fields(data,'Habitat')


BRIT118134 100 0 1
BRIT118134 100 0 2
BRIT118134 100 1 2
BRIT118051 100 0 1
BRIT118051 100 0 2
BRIT118051 100 1 2
BRIT118141 82 0 1
BRIT118141 91 0 2
BRIT118141 100 1 2
BRIT118116 100 0 1
BRIT118116 100 0 2
BRIT118116 100 1 2
BRIT118093 100 0 1
BRIT118093 100 0 2
BRIT118093 100 1 2
BRIT118000 100 0 1
BRIT118000 100 0 2
BRIT118000 100 1 2
BRIT118119 100 0 1
BRIT118119 86 0 2
BRIT118119 86 1 2
BRIT118145 100 0 1
BRIT118145 0 0 2
BRIT118145 0 1 2
BRIT118181 0 0 1
BRIT118181 0 0 2
BRIT118181 100 1 2
BRIT118152 100 0 1
BRIT118152 100 0 2
BRIT118152 100 1 2
BRIT118039 100 0 1
BRIT118039 100 0 2
BRIT118039 100 1 2
BRIT118007 100 0 1
BRIT118007 100 0 2
BRIT118007 100 1 2
BRIT118130 0 0 1
BRIT118130 99 0 2
BRIT118130 0 1 2
BRIT118029 0 0 1
BRIT118029 0 0 2
BRIT118029 0 1 2
BRIT118087 100 0 1
BRIT118087 100 0 2
BRIT118087 100 1 2
BRIT118155 100 0 1
BRIT118155 100 0 2
BRIT118155 100 1 2
BRIT118105 100 0 1
BRIT118105 100 0 2
BRIT118105 100 1 2
BRIT118103 100 0 1
BRIT118103 100 0 2
BRIT118103 100 1